# 1. Chuẩn bị thư viện

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import numpy as np
import faiss
import json
from torch.nn.utils.rnn import pad_sequence
import random
import math
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:

print("Phiên bản PyTorch:", torch.__version__)
print("Phiên bản CUDA mà PyTorch sử dụng:", torch.version.cuda)
print("CUDA khả dụng:", torch.cuda.is_available())

Phiên bản PyTorch: 2.5.1
Phiên bản CUDA mà PyTorch sử dụng: 12.1
CUDA khả dụng: True


# 2. Áp dụng kỹ thuật DAPT
Xây dựng thêm 1 block MLM huấn luyện lại qua bộ dữ liệu đặc thù đã chuẩn bị trước

In [ ]:
class RawDataset(Dataset):
    def __init__(self, lines, tokenizer, max_length=256, mlm_prob=0.15):
        self.lines = lines
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mlm_prob = mlm_prob

    def __len__(self):
        return len(self.lines)

    def mask_tokens(self, inputs, special_tokens_mask):
        labels = inputs.clone()
        probability_matrix = torch.full(labels.shape, self.mlm_prob)
        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)

        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100

        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        return inputs, labels

    def __getitem__(self, idx):
        line = self.lines[idx]
        encoding = self.tokenizer(
            line,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_special_tokens_mask=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        special_tokens_mask = encoding['special_tokens_mask'].squeeze(0).bool()
        input_ids, labels = self.mask_tokens(input_ids.clone(), special_tokens_mask)
        attention_mask = encoding['attention_mask'].squeeze(0)
        token_type_ids = torch.zeros(self.max_length, dtype=torch.long)  # PhoBERT không dùng token_type_ids nhưng model yêu cầu

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "token_type_ids": token_type_ids,
        }

class EarlyStopping:
    def __init__(self, patience=2, verbose=True, save_path="/content/MyDrive/models/my_model"):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = float("inf")
        self.counter = 0
        self.save_path = save_path

    def __call__(self, val_loss, model, tokenizer):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            if self.verbose:
                print(f"→ Val loss improved, saving model to {self.save_path}")
            self.save_checkpoint(model, tokenizer)
        else:
            self.counter += 1
            if self.verbose:
                print(f"→ No improvement. EarlyStopping counter: {self.counter}/{self.patience}")
        return self.counter >= self.patience

    def save_checkpoint(self, model, tokenizer):
        model.save_pretrained(self.save_path)
        tokenizer.save_pretrained(self.save_path)


# --- 2. Load dữ liệu ---
with open("data/legal_text.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

split_idx = int(0.8 * len(lines))
train_lines = lines[:split_idx]
val_lines = lines[split_idx:]

# --- 3. Khởi tạo tokenizer, model, dataset, dataloader ---
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForMaskedLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_dataset = RawDataset(train_lines, tokenizer)
val_dataset = RawDataset(val_lines, tokenizer)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


# --- 4. Hàm train 1 epoch ---
def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    loop = tqdm(dataloader, desc="Training")

    for batch in loop:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            token_type_ids=token_type_ids,
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader)
    return avg_loss


# --- 5. Hàm validation ---
def validate(model, dataloader, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                token_type_ids=token_type_ids,
            )
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity


# --- 6. Vòng lặp huấn luyện chính ---
early_stopping = EarlyStopping(patience=2, verbose=True, save_path="models/my_model")

num_epochs = 5

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}")

    val_loss, val_perplexity = validate(model, val_loader, device)
    print(f"Validation Loss: {val_loss:.4f}, Perplexity: {val_perplexity:.2f}")

    if early_stopping(val_loss, model, tokenizer):
        print("Early stopping triggered. Training stopped.")
        break




Kiểm tra tác vụ mask

In [ ]:
from transformers import pipeline

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

print(fill_mask("Công dân có quyền <mask> tại nơi cư trú."))

Device set to use cuda:0


[{'score': 0.21961596608161926, 'token': 235, 'token_str': 'sống', 'sequence': 'Công dân có quyền sống tại nơi cư trú.'}, {'score': 0.21799877285957336, 'token': 25, 'token_str': 'ở', 'sequence': 'Công dân có quyền ở tại nơi cư trú.'}, {'score': 0.15344639122486115, 'token': 385, 'token_str': 'tự', 'sequence': 'Công dân có quyền tự tại nơi cư trú.'}, {'score': 0.10251910239458084, 'token': 1354, 'token_str': 'trú', 'sequence': 'Công dân có quyền trú tại nơi cư trú.'}, {'score': 0.032492998987436295, 'token': 5157, 'token_str': 'cư_trú', 'sequence': 'Công dân có quyền cư_trú tại nơi cư trú.'}]


In [ ]:
from transformers import pipeline

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

text = "Người chưa <mask> muốn <mask> thường trú phải có sự <mask> của cha mẹ hoặc người giám hộ."

results = fill_mask(text)

# In ra từng dòng dự đoán cho mỗi vị trí <mask>
for i, mask_predictions in enumerate(results):
    print(f"\nDự đoán cho <mask> thứ {i+1}:")
    for pred in mask_predictions:
        token = pred["token_str"]
        score = pred["score"]
        print(f"  {token:<15} (score: {score:.4f})")


Device set to use cuda:0



Dự đoán cho <mask> thứ 1:
  thành_niên      (score: 0.7145)
  thành           (score: 0.1371)
  chồng           (score: 0.0640)
  chết            (score: 0.0205)
  vợ              (score: 0.0075)

Dự đoán cho <mask> thứ 2:
  đến             (score: 0.2873)
  đăng            (score: 0.2173)
  ký              (score: 0.1525)
  đi              (score: 0.0890)
  chuyển          (score: 0.0396)

Dự đoán cho <mask> thứ 3:
  đồng_ý          (score: 0.5017)
  cho_phép        (score: 0.0787)
  chấp_thuận      (score: 0.0570)
  kiến            (score: 0.0317)
  phép            (score: 0.0211)


# 2. Trích xuất văn bản luật liên quan
Cho mô hình đã DAPT thực hiện tìm kiếm các đoạn văn bản Điều luật liên quan tới câu hỏi nhất

## 2.1. Tiền xử lý dữ liệu

### a) Dữ liệu câu hỏi

In [1]:
def preprocess_qa(input_path="data/qa.csv", output_path="data/clean_qa.csv"):
    """
    Xử lý đơn giản dữ liệu câu hỏi: chỉ chuyển về lowercase.
    """
    df = pd.read_csv(input_path)

    # Chuyển câu hỏi về chữ thường
    df["clean_question"] = df["question"].astype(str).str.strip().str.lower()

    # Bỏ dòng có nội dung trống
    df = df[df["clean_question"].str.len() > 0]

    df.to_csv(output_path, index=False)
    print(f"✅ Đã lưu {len(df)} câu hỏi vào {output_path}")

preprocess_qa()

NameError: name 'pd' is not defined

### b) Dữ liệu văn bản pháp luật

In [ ]:
def preprocess_law(input_path="data/legal_data.csv", output_path="data/law_articles.csv"):
    """
    Xử lý đơn giản dữ liệu Điều luật: gộp các trường nội dung và chuyển về lowercase.
    """
    df = pd.read_csv(input_path)

    # Gộp: Điều + Tên điều + Nội dung
    def merge(row):
        parts = [
            "Điều",
            str(row["điều"]).strip() if pd.notna(row["điều"]) else "",
            str(row["tên điều"]).strip() if pd.notna(row["tên điều"]) else "",
            str(row["nội dung"]).strip() if pd.notna(row["nội dung"]) else "",
        ]
        return f"{parts[0]} {parts[1]}. {parts[2]}. {parts[3]}".strip()

    df["content"] = df.apply(merge, axis=1)

    # Lưu cột cần thiết
    df_out = df[["id", "content"]].copy()
    df_out.to_csv(output_path, index=False)

    print(f"✅ Đã lưu {len(df_out)} Điều luật vào {output_path}")

preprocess_law()

✅ Đã lưu 90 Điều luật vào /content/drive/MyDrive/data/law_articles.csv


## 2.2. Truy xuất top-k đoạn văn bản liên quan

Load mô hình đã DAPT

In [ ]:
def load_model(model_path="models/my_model"):
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
    model = AutoModel.from_pretrained(model_path)
    model.eval()
    return tokenizer, model

tokenizer, model = load_model("models/my_model")

Some weights of RobertaModel were not initialized from the model checkpoint at models/my_model and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Hàm encode

In [ ]:
def encode_text_attention(texts, tokenizer, model, device='cuda', max_length=256):
    model.to(device)
    model.eval()

    inputs = tokenizer(
        texts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=max_length
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden = outputs.last_hidden_state  # (batch_size, seq_len, hidden_dim)
        attention_mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden.size()).float()  # (batch, seq_len, hidden_dim)

        weighted_sum = (last_hidden * attention_mask).sum(dim=1)  # sum token embeddings weighted by mask
        valid_token_count = attention_mask.sum(dim=1)  # (batch_size, hidden_dim)
        embeddings = weighted_sum / valid_token_count  # mean pooling with mask

    return embeddings.cpu().numpy()  # (batch_size, hidden_dim)

def encode_text_cls(text, tokenizer, model):
    text = text.lower().strip()
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()

Xây dựng FAISS index

In [ ]:
def build_faiss_index_attention(texts, tokenizer, model, batch_size=32, device='cuda'):
    """
    Xây dựng FAISS index từ các đoạn văn bản
    """
    vectors = []
    for i in tqdm(range(0, len(texts), batch_size), desc="🔧 Encoding điều luật"):
        batch = texts[i:i + batch_size]
        embeddings = encode_text_attention(batch, tokenizer, model, device=device)
        vectors.append(embeddings)
    vectors = np.concatenate(vectors, axis=0).astype("float32")
    faiss.normalize_L2(vectors)
    dim = vectors.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(vectors)
    return index, vectors

def build_faiss_index_cls(texts, tokenizer, model):
    vectors = []
    for t in tqdm(texts, desc="🔧 Encoding điều luật"):
        vectors.append(encode_text_cls(t, tokenizer, model))
    vectors = np.stack(vectors).astype("float32")
    faiss.normalize_L2(vectors)
    dim = vectors.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(vectors)
    return index, vectors

Truy xuất top-k

In [ ]:
def retrieve_top_k_attention(index, law_texts, questions, tokenizer, model, k):
    results = []
    for q in tqdm(questions, desc="🔍 Truy xuất câu hỏi"):
        q_vec = encode_text_attention(q, tokenizer, model).astype("float32").reshape(1, -1)
        faiss.normalize_L2(q_vec)
        D, I = index.search(q_vec, k)
        matched = [law_texts[i] for i in I[0]]
        scores = D[0].tolist()
        results.append({
            "question": q,
            "top_k_laws": matched,
            "scores": scores
        })
    return results

def retrieve_top_k_cls(index, law_texts, questions, tokenizer, model, k):
    results = []
    for q in tqdm(questions, desc="🔍 Truy xuất câu hỏi"):
        q_vec = encode_text_cls(q, tokenizer, model).astype("float32").reshape(1, -1)
        faiss.normalize_L2(q_vec)
        D, I = index.search(q_vec, k)
        matched = [law_texts[i] for i in I[0]]
        scores = D[0].tolist()
        results.append({
            "question": q,
            "top_k_laws": matched,
            "scores": scores
        })
    return results


Rerank trên câu trả lời

In [ ]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
def rerank(question, top_k_laws, cross_encoder, top_n):
    # Tạo danh sách các cặp (query, document)
    pairs = [(question, law) for law in top_k_laws]

    # Dự đoán điểm tương đồng cho từng cặp
    scores = cross_encoder.predict(pairs)

    scores = [float(score) for score in scores]

    # Sắp xếp theo điểm giảm dần
    sorted_pairs = sorted(zip(top_k_laws, scores), key=lambda x: x[1], reverse=True)

    # Trả về top_n kết quả
    reranked_laws = [law for law, _ in sorted_pairs[:top_n]]
    reranked_scores = [score for _, score in sorted_pairs[:top_n]]

    return reranked_laws, reranked_scores


Lưu vector

In [ ]:
df_law = pd.read_csv("data/law_articles.csv")
df_qa = pd.read_csv("data/clean_qa.csv")

law_texts = df_law["content"].tolist()
questions = df_qa["clean_question"].astype(str).tolist()

print("Load xong")

index, _ = build_faiss_index_attention(law_texts, tokenizer, model)


Load xong


🔧 Encoding điều luật: 100%|██████████| 3/3 [00:01<00:00,  2.92it/s]


Pipeline chính

In [ ]:
def pipeline_retrieve_and_rerank(question, index, law_texts, tokenizer, model, cross_encoder, top_k=30, top_n=5):
    results = retrieve_top_k_attention(index, law_texts, [question], tokenizer, model, k=top_k)
    top_k_laws = results[0]["top_k_laws"]
    reranked_laws, reranked_scores = rerank(question, top_k_laws, cross_encoder, top_n=top_n)
    return reranked_laws, reranked_scores

Chạy kiếm tra 1 câu hỏi đầu vào

In [ ]:
question = "Tôi đi Singapore 10 ngày có phải khai báo tạm vắng không?"

top5_laws, top5_scores = pipeline_retrieve_and_rerank(
    question,
    index,
    law_texts,
    tokenizer,
    model,
    cross_encoder,
    top_k=30,
    top_n=5
)

print(f"Câu hỏi: {question}")
print("Top 5 đoạn luật đã rerank:")
for i, (law, score) in enumerate(zip(top5_laws, top5_scores), 1):
    print(f"{i}. (Điểm: {score:.4f}) {law[:300]}{'...' if len(law) > 300 else ''}")

🔍 Truy xuất câu hỏi: 100%|██████████| 1/1 [00:00<00:00, 35.25it/s]


Câu hỏi: Tôi đi Singapore 10 ngày có phải khai báo tạm vắng không?
Top 5 đoạn luật đã rerank:
1. (Điểm: 3.5183) Điều 4. Nơi cư trú của người không có nơi thường trú, nơi tạm trú. 1. Người không có nơi thường trú, nơi tạm trú phải khai báo ngay thông tin về cư trú với cơ quan đăng ký cư trú tại nơi ở hiện tại. Trường hợp qua kiểm tra, rà soát, cơ quan đăng ký cư trú phát hiện người thuộc trường hợp phải khai b...
2. (Điểm: 3.1157) Điều 4. Nơi cư trú của người không có nơi thường trú, nơi tạm trú. 1. Người không có nơi thường trú, nơi tạm trú khai báo thông tin về cư trú theo mẫu Tờ khai thay đổi thông tin cư trú và nộp trực tuyến, trực tiếp hoặc qua dịch vụ bưu chính công ích đến cơ quan đăng ký cư trú tại nơi ở hiện tại theo...
3. (Điểm: 1.9634) Điều 19. Nơi cư trú của người không có nơi thường trú, nơi tạm trú. 1. Nơi cư trú của người không có cả nơi thường trú và nơi tạm trú do không đủ điều kiện đăng ký thường trú, đăng ký tạm trú là nơi ở hiện tại của người đó; trường hợp không có 

# 3. Mô hình sinh câu trả lời

In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("API_KEY"))

generative_model = genai.GenerativeModel("gemini-1.5-flash")  # hoặc "gemini-1.5-pro"

question = "Thủ tục đăng ký tạm trú là gì? Đối tượng nào phải đăng ký tạm trú?"

top5_laws, top5_scores = pipeline_retrieve_and_rerank(
    question,
    index,
    law_texts,
    tokenizer,
    model,
    cross_encoder,
    top_k=30,
    top_n=5
)

law_context = "\n".join(f"- {law}" for law in top5_laws)

prompt = f"""
Bạn là trợ lý ảo pháp lý của chính quyền Việt Nam. Hãy trả lời câu hỏi của người dân bằng cách dựa vào các văn bản pháp luật sau:

{law_context}

❓ Câu hỏi: {question}

💬 Trả lời (ngắn gọn, rõ ràng, theo luật): 
"""

response = generative_model.generate_content(prompt)

# In ra câu hỏi, câu trả lời và các văn bản pháp luật liên quan ra 1 file reponse.txt
with open("response.txt", "w", encoding="utf-8") as f:
    f.write("🔍 Câu hỏi: " + question + "\n")
    f.write("💬 Trả lời: " + response.text + "\n")
    f.write("📜 Các văn bản pháp luật liên quan:\n")
    f.write(law_context + "\n")


🔍 Truy xuất câu hỏi: 100%|██████████| 1/1 [00:00<00:00, 38.10it/s]
